# Reconnaissance d'entités nommées avec SpaCy

Preparation et chargement du corpus

In [1]:
from pathlib import Path
import re


data_dir = Path("../../data/txt")

all_txt = list(data_dir.glob("*.txt"))

YEAR = "1950"
pat = re.compile(rf"{YEAR}")

files_year = [p for p in all_txt if pat.search(p.name)]
print(f"Fichiers trouvés pour {YEAR}: {len(files_year)}")

corpus_year = ""
for path in files_year:
    for enc in ("utf-8", "latin-1", "cp1252"):
        try:
            with open(path, "r", encoding=enc, errors="ignore") as f:
                corpus_year += f.read() + "\n"
            break
        except Exception:
            continue

print(f"Taille du corpus {YEAR}: {len(corpus_year):,} caractères")
print("\nExtrait:\n", corpus_year[:500])

Fichiers trouvés pour 1950: 4
Taille du corpus 1950: 133,895 caractères

Extrait:
 L'AVENIR DU LUXEMBOURG Samedi 15 avri j 350, 
MORHET 
Soirée dramatique 
1 Le cercle dramatique Sainte-Cécile 
de Morhet reprendra, ce dimanche 16 
avril ^Quasimodo), sa brillante soirée 
qui a remporté un succès si remarqua-
| bie le 10 mars dernier. 
i Rappelons ie programme : 
; 1) ouverture : « Brabançonne »,par 
• la Fantare ; 2) « La .bohème », chœur 
à 2 voix exécuté par JV^.es Renée Cara, 
j Josée Goffin, Anyse Hubermont et Hé-
f lène Bellanger ; a) La comédie en deux 
actes de Marcell* 


La documentation est accessible ici: https://spacy.io/api

## Imports

In [2]:
from collections import defaultdict
import spacy
from spacy.lang.fr.examples import sentences
!python -m spacy download fr_core_news_md

ModuleNotFoundError: No module named 'spacy'

In [4]:
nlp = spacy.load('fr_core_news_md')

## Exemple sur un corpus de test fourni par SpaCy

In [ ]:
# Imprimer le corpus de Spacy
sentences

In [ ]:
# Isoler la première phrase
sent = sentences[0]
sent

In [ ]:
# Traiter la phrase avec Spacy
doc = nlp(sent)

In [ ]:
type(doc)

In [ ]:
doc.text

In [ ]:
doc.to_json()

In [ ]:
# Appliquer le test sur toutes les phrases
for sent in sentences:
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}")
    else:
        print(f"'{doc.text}' ne contient aucune entité")

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [ ]:
# Charger le texte
n=1000000
text = open("../data/all.txt", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus